In [1]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
from IPython.display import display
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
class SHRS_PS:
    def __init__(self, path):
        self.path = path
        self.Data = pd.read_csv(path)
        display(self.Data.head())
        self.Min = np.min(self.Data['sugar'])
        self.Max = np.max(self.Data['sugar'])
    
    def fit(self):
        """Data: Pandas Dataframe"""
        self.normalize()
        display(self.Data.head())
        y = self.Data['label']
        x = self.Data.drop(["label"], axis=1).to_numpy()
        xTrain, yTrain = x[:80], y[:80]
        xTest, yTest = x[80:], y[80:]
        self.clf = GradientBoostingClassifier(n_estimators=100, learning_rate=.01, max_depth=1, random_state=0)
        self.clf.fit(xTrain, yTrain)
        print("Model Trained\nAccuracy : {}".format(self.clf.score(xTest, yTest)))        
    
    def single(self, num):
        return num;
    
    def normalize(self, single=0):
        """Data : Dataframe"""
        scaler = MinMaxScaler()
        x = np.reshape(self.Data['sugar'].to_numpy(), (-1,1))
        scaler = scaler.fit(x)
        if(single==0):
            sugar_normalised = scaler.transform(x)
            self.Data['sugar'] = sugar_normalised
            print("Executed Zero")
        else:
            print(single, self.Min, self.Max)
            return np.array((single - self.Min) / (self.Max - self.Min))
        
        
    def predict(self, sugar, insulin, exercise, diet,  severe_disease):
        print(self.clf.predict(np.reshape([self.normalize(sugar), insulin, exercise, diet, severe_disease], (1,-1))))

In [3]:
# shrs = SHRS_PS("./data_raka.csv")

,sugar,insulin,exercise,diet,severe_disease,label
0,337,1,0,1,0,3
1,312,1,1,1,1,3
2,322,0,1,1,0,3
3,152,0,0,1,1,1
4,310,0,0,1,0,3


In [4]:
# shrs.fit()

Executed Zero


,sugar,insulin,exercise,diet,severe_disease,label
0,0.836257,1,0,1,0,3
1,0.763158,1,1,1,1,3
2,0.792398,0,1,1,0,3
3,0.295322,0,0,1,1,1
4,0.757310,0,0,1,0,3


Model Trained
Accuracy : 0.95


In [5]:
# shrs.predict(299, 1, 1, 1, 1)

299 51 393
[2]
